In [1]:
#done in google colab as I was unable to do it in jupyter

import numpy as np
import pickle
import os
import download
from dataset import one_hot_encoded
import cifar10


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks'
/content


In [3]:
cifar10.data_path = "data/CIFAR-10/Desktop"

In [4]:
cifar10.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [5]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [6]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/Desktop/cifar-10-batches-py/test_batch


In [7]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [8]:
Xtrain = images_train
Xtrain.shape

(50000, 32, 32, 3)

In [9]:
Ytrain = cls_train
Ytrain.shape

(50000,)

In [14]:
#we will reshape Xtrain 32*32*3 = 3072
Xtrain = Xtrain.reshape(50000, 3072)
Xtrain.shape

(50000, 3072)

In [11]:
Xtest = images_test.reshape(10000, 3072)
Ytest = cls_test


In [15]:
#applying pca
pca = PCA()
pca.fit(Xtrain)

PCA()

In [16]:
#finding optimal number of components
k = 0
totalSum = sum(pca.explained_variance_)
currentSum = 0

while currentSum/totalSum < 0.95:
    currentSum += pca.explained_variance_[k]
    k+= 1
k

217

In [18]:
#optimising pca with n_components as k
pca_2 = PCA(n_components = k, whiten = True)
Xtrain_pca = pca_2.fit_transform(Xtrain)
Xtest_pca = pca_2.transform(Xtest)

In [19]:
#we will apply support vector classifier
clf = SVC()
clf.fit(Xtrain_pca, Ytrain)

SVC()

In [20]:
#we wil predict Y
Ypred = clf.predict(Xtest_pca)


In [21]:
# we will convert Ypred into class names since we want output in class names
class_name_pred = []
for i in Ypred:
      class_name_pred.append(class_names[i])

class_name_pred = np.array(class_name_pred)

In [22]:
#we will save it in csv file
np.savetxt('Cifar_pred.csv', class_name_pred, fmt = '%s', delimiter = ',', encoding = None)

In [23]:
#calculating score
clf.score(Xtest_pca, Ytest)

0.555